In [2]:
import psycopg2
from dotenv import load_dotenv
import pandas as pd
from phone_number import padronize_phone
from models import Empresa

load_dotenv()

c:\Projetos\empresas\empresas\phone_number.py:24: SyntaxWarning: invalid escape sequence '\d'
  busca_falta_55_e_9 = re.search('^(\d{2})(\d{8})$', remove_caracteres)


ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [2]:
def connect():
    """Connect to the database"""
    conn = None
    try:
        print('Connecting...')
        conn = psycopg2.connect(
            host="localhost",
            database='Dados_RFB',
            user='postgres',
            password='postgres'
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print('All good, Connection successful!')   
    return conn

In [3]:
def sql_to_dataframe(conn, query, column_names):
    """Import data from a PostgreSQL database using a SELECT query"""

    cursor = conn.cursor()
    try:
        cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    tuples_list = cursor.fetchall()
    cursor.close()
    df = pd.DataFrame(tuples_list, columns=column_names)
    return df


In [6]:
query = """
    select est.cnpj_basico, est.cnpj_ordem, est.cnpj_dv, natju.descricao, e.razao_social,est.nome_fantasia, cnae.descricao, est.tipo_logradouro, est.logradouro, est.numero, est.complemento, est.bairro, est.cep, est.uf, est.ddd_1, est.telefone_1, est.correio_eletronico
    from estabelecimento as est, empresa as e, cnae, natju, simples as s
    where est.municipio = '2531'
    and e.cnpj_basico = est.cnpj_basico
    and e.natureza_juridica = '2135'
    and cnae.codigo = est.cnae_fiscal_principal
    and est.situacao_cadastral = '02'
    and e.natureza_juridica = natju.codigo
    and s.cnpj_basico = e.cnpj_basico
    and s.opcao_mei = 'S'
"""
column_names = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'natju_descricao', 'razao_social', 'nome_fantasia', 'cnae_descricao', 'tipo_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'ddd_1', 'telefone_1', 'correio_eletronico']
conn = connect()
df = sql_to_dataframe(conn, query, column_names)
conn.close()
df.head()

Connecting...
All good, Connection successful!


,cnpj_basico,cnpj_ordem,cnpj_dv,natju_descricao,razao_social,nome_fantasia,cnae_descricao,tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,ddd_1,telefone_1,correio_eletronico
0,40040077,0001,00,Empresário (Individual),DANIELE LEITAO DE ALBUQUERQUE 66530717404,None,Comércio varejista de artigos do vestuário e a...,RUA,PEDRO CELSO,53,None,CAMPO GRANDE,52031180,PE,81,97129909,GERONIMOBERJAN@HOTMAIL.COM
1,40051472,0001,98,Empresário (Individual),NILSON ESTEVAM DA SILVA 23351292449,None,Promoção de vendas,RUA,PADRE DIOGO RODRIGUES,109,CASA 06,BARRO,50900110,PE,81,84540429,None
2,36865736,0001,70,Empresário (Individual),CARLOS FERNANDO MARTINS FERREIRA 09282166406,None,"Lanchonetes, casas de chá, de sucos e similares",RUA,TENENTE DOMINGOS DE BRITO,SN,None,BOA VIAGEM,51021100,PE,81,91359073,None
3,40065634,0001,47,Empresário (Individual),40.065.634 WAGNER TEIXEIRA DE OLIVEIRA,None,Outras atividades de serviços prestados princi...,RUA,VINTE E QUATRO DE MAIO - ATE 209/210,22,ANDAR 1,SANTO ANTONIO,50010025,PE,81,85751790,WAGNERTEIIXEIR@GMAIL.COM
4,40069185,0001,05,Empresário (Individual),RENATO NASCIMENTO DE SANTANA 08419734403,None,Reparação e manutenção de computadores e de eq...,RUA,ALVENOPOLIS,206,None,JARDIM SAO PAULO,50790200,PE,81,87731651,RENATOSANTANA88@HOTMAIL.COM


In [9]:
def create_empresa(data):
    cnpj = data['cnpj_basico']+ '/' + data['cnpj_ordem'] + '-' + data['cnpj_dv']
    endereco = f"{data['tipo_logradouro']} {data['logradouro']}"
    if data['numero'] != '' or data['numero'] != None:
        endereco += f", Nº {data['numero']}"
    telefone = padronize_phone(f"{data['ddd_1']}{data['telefone_1']}")
    Empresa.objects.create(
        cnpj=cnpj,
        natju=data['natju_descricao'],
        razao_social=data['razao_social'],
        nome_fantasia=data['nome_fantasia'],
        cnae_descricao=data['cnae_descricao'],
        endereco=endereco,
        bairro=data['bairro'],
        telefone=telefone,
        email=data['correio_eletronico'],
    )

In [10]:
for index, row in df.iterrows():
    razao_social = row['razao_social']
    create_empresa(row)

NameError: name 'padronize_phone' is not defined